# Imports

In [1]:
import pandas as pd
from IPython.display import display, Markdown
import numpy as np
import lda
import nltk
from nltk.corpus import words
from nltk.metrics.distance import jaccard_distance, edit_distance
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from matplotlib import pyplot as plt
import seaborn as sns
import datetime

nltk.download('words')
pd.set_option('display.max_columns', None)

C:\Users\bck00\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\bck00\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


# Assignment Overview

In [2]:
# Note: requires "Export Embedded HTML" jupyter extension"to be enabled
with open('../references/README.md', 'r') as fh:
    content = fh.read()

display(Markdown(content))

# Business Problem:
The finance team needs to segment prospective users (level). This classification will be used to determine deposit/withdrawal limits. How would you build a model and how would you producitonize it?

## Submissions:
* A pickled model file
* The code used to generate the model
* Production design summary
    * please demonstrate your thought thought process for how you would put this into production/why you made these design decision, i.e.
        * how would you implement this in a cloud (AWS or GCP) architecture?
        * which IaaS/PaaS components would you use?
        * why would you use these?
        * how would you monitor this service in production to accommodate for reliability?
        * what business considerations might impact your design decisions?
        * etc.

## How you will be evaluated:
* We will evaluate your work based on code quality, problem-solving, workflow, analytical rigor, and ability to communicate your work

## Additional Notes:

* At Gemini. we prefer python/jupter for analysis, but if you feel more comfortable using another tool/language, please feel free
* This is an open-ended question, so we are not looking for 100% accuracy out of your model. We're looking for you to show off your thought process, and have fun coding!

# Dictionary

### Demographics table:
this table contains contextual information about a sample of gemini users.
the target class we are trying to predict is LEVEL

* EXCHANGE_ACCOUNT_ID - unique identifier for a person
* level - the target class we are trying to predict
* CREATED_AT - date the account was created
* FIRST_VERIFIED_AT - date the account was verified
* STATE_CODE - two letter state that the user of the account resides in
* COUNTRY_CODE - two letter country code the user resides in
* BIRTH_YEAR - year the user of the account was born
* OCCUPATION - stated occupation of the user
* BROWSER_COUNT - number of distinct browsers user has used
* OS_COUNT - number of distinct operating systems detected
* TOTAL_DEVICES - total count of distinct devices used by the account
* SESSION_COUNT - count of sessions initiated by the user
* COUNT_BANKS - count of bank accounts attached to user

### Exposure table:
this table contains aggregated information about the source and destination of user's transfers

* EXCHANGE_ACCOUNT_ID - unique identifier for a person
* SENT_INDIRECT_EXPOSURE - total outgoing volume (multiple hops before arriving at destination) to specified cluster
* SENT_DIRECT_EXPOSURE - direct direct send volume to specified cluster
* RECEIVED_INDIRECT_EXPOSURE - total incoming volume (multiple hops before arriving at Gemini) from specified cluster
* RECEIVED_DIRECT_EXPOSURE - total receiving volume (direct receive) from specified cluster
* cluster_category - type of service (ie exchange, merchant, private wallet etc)
* cluster_name - name of the service

### Transfers table:
this table contains a sample of transfer logs in 2020

* ACCOUNT_ID - unique identifier for a person
* TX_TIME - timestamp of a transfer
* AMOUNT - quantity transfered
* currency - currency of the transfer
* tx_type - type of the transfer (deposit/withdrawal/etc)

# EDA

In [3]:
def eda(df):
    """Print exploratory statistics about given dataframe."""
    
    print('Shape:')
    print(df.shape)
    print('-----------------------------')
    print('Sample:')
    print(df.head(3).append(df.tail(3)))
    print('-----------------------------')
    print('Describe:')
    print(df.describe(include='all'))
    print('-----------------------------')
    print('Percent missing values:')
    print(df.isnull().sum() / df.shape[0] * 100.00)
    print('-----------------------------')
    print('Data types:')
    print(df.dtypes)
    
    return None

In [4]:
def clean_text_columns(df, col_list=None):
    """Basic text cleaning."""
    
    # use object columns if none provided
    if col_list is None:
        col_list = df.loc[:, df.dtypes == object].columns
        
    # get rid of non letter characters
    df[col_list] = df[col_list].apply(lambda x: x.str.replace('[^a-zA-Z ]', ''))    
    # convert to all uppercase
    df[col_list] = df[col_list].apply(lambda x: x.astype(str).str.upper())
    # strip whitespace
    df[col_list] = df[col_list].apply(lambda x: x.str.strip())
    
    return df

In [5]:
def autocorrect(myword):
    """Correct mispelled words using levanstine distance."""
    
    correct_spellings = words.words()
    distance_dict = {}
    for word in correct_spellings:
        ed = nltk.edit_distance(myword, word)
        distance_dict[word] = ed
        if ed == 0:
            break

    return(min(distance_dict, key=distance_dict.get))

In [6]:
def group_occupation(occupation, other=None):
    """Narrow down occupations."""

    tech_keywords = ['ENGINEER', 'IT', 'SOFTWARE', 'TECHNOLOGY', 'DATA', 'DEVELOPER', 'PROGRAMMER', 'TECH', 'WEB', 'COMPUTER', 
                        'DBA', 'SCIENTIST', 'INFO']
    law_keywords = ['LAWYER', 'ATTORNEY', 'LEGAL']
    medicine_keywords = ['PHYSICIAN', 'DOCTOR', 'SURGEON', 'DENTIST', 'PHARMACIST', 'MD', 'NURSE', 'RN', 'NURSING', 'MEDICAL']
    self_keywords = ['SELF']
    sales_keywords = ['SALES']
    retired_keywords = ['RETIRED', 'RETIREE', 'RETIRE']
    student_keywords = ['STUDENT']
    executive_keywords = ['CEO', 'CTO', 'COO', 'CPO', 'CFO', 'CIO', 'DIRECTOR', 'FOUNDER', 'PRESIDENT', 'EXECUTIVE', 'OWNER', 'VP']
    unemployed_keywords = ['UNEMPLOYED', 'NONE']
    finance_keywords = ['FINANCE', 'ACCOUNTANT', 'WEALTH', 'BANKER', 'BANK', 'FINANCIAL', 'CPA', 'ACCOUNTING']
    manager_keywords = ['MANAGER', 'MANAGEMENT', 'SUPERVISOR']
    real_estate_keywords = ['REAL ESTATE', 'REALTOR', 'BROKER']
    business_keywords = ['BUSINESS', 'BUSINESSMAN']
    if other:
        if occupation in other:
            return 'OTHER'
        else:
            return occupation
    else:
        if any(word in occupation for word in tech_keywords):
            return 'TECH'
        elif any(word in occupation for word in law_keywords):
            return 'LAW'
        elif any(word in occupation for word in medicine_keywords):
            return 'MEDICINE'    
        elif any(word in occupation for word in self_keywords):
            return 'SELF'
        elif any(word in occupation for word in sales_keywords):
            return 'SALES'   
        elif any(word in occupation for word in retired_keywords):
            return 'RETIRED'   
        elif any(word in occupation for word in student_keywords):
            return 'STUDENT'  
        elif any(word in occupation for word in executive_keywords):
            return 'EXECUTIVE'  
        elif any(word in occupation for word in finance_keywords):
            return 'FINANCE'  
        elif any(word in occupation for word in real_estate_keywords):
            return 'REAL ESTATE' 
        elif any(word in occupation for word in manager_keywords):
            return 'MANAGER'  
        elif any(word in occupation for word in business_keywords):
            return 'BUSINESS'  
        elif any(word in occupation for word in unemployed_keywords):
            return 'UNEMPLOYED'  
        elif occupation == 'NAN':
            return 'MISSING'
        else:
            return occupation

In [7]:
data_dir = '../data/raw/'

### Demographics

In [8]:
dem = pd.read_csv(data_dir + 'demographics_train.csv')

In [9]:
eda(dem)

Shape:
(8241, 13)
-----------------------------
Sample:
      EXCHANGE_ACCOUNT_ID    level    CREATED_AT FIRST_VERIFIED_AT STATE_CODE  \
0                       1  level_a        1/0/00     12/14/16 0:00        NaN   
1                       3  level_a        1/0/00      4/10/17 0:00        NaN   
2                       5  level_a        1/0/00       5/2/17 0:00        NaN   
8238                21663  level_b  10/5/15 0:00      10/5/15 0:00         NY   
8239                21665  level_b   6/4/19 0:00       6/4/19 0:00         CA   
8240                21667  level_b  9/17/19 0:00      9/16/19 0:00         GA   

     COUNTRY_CODE  BIRTH_YEAR OCCUPATION  BROWSER_COUNT  OS_COUNT  \
0             NaN         NaN        NaN            NaN       NaN   
1             NaN         NaN        NaN            NaN       NaN   
2             NaN         NaN        NaN            NaN       NaN   
8238           us      1982.0    finance            1.0       1.0   
8239           us      1984.0  

In [10]:
dem = clean_text_columns(dem, col_list=['STATE_CODE', 'COUNTRY_CODE', 'OCCUPATION'])

In [11]:
# autocorrect words which may be mispelled
# dem['OCCUPATION0'] = dem['OCCUPATION'].apply(autocorrect)
# dem.OCCUPATION0.value_counts()

In [12]:
# dem[dem['OCCUPATION'] != dem['OCCUPATION0']]

In [13]:
dem.head(2).append(dem.tail(2))

,EXCHANGE_ACCOUNT_ID,level,CREATED_AT,FIRST_VERIFIED_AT,STATE_CODE,COUNTRY_CODE,BIRTH_YEAR,OCCUPATION,BROWSER_COUNT,OS_COUNT,TOTAL_DEVICES,SESSION_COUNT,COUNT_BANKS
0,1,level_a,1/0/00,12/14/16 0:00,NAN,NAN,NaN,NAN,NaN,NaN,NaN,1,6.0
1,3,level_a,1/0/00,4/10/17 0:00,NAN,NAN,NaN,NAN,NaN,NaN,NaN,1,4.0
8239,21665,level_b,6/4/19 0:00,6/4/19 0:00,CA,US,1984.0,NAN,3.0,1.0,5.0,134,1.0
8240,21667,level_b,9/17/19 0:00,9/16/19 0:00,GA,US,1977.0,NAN,4.0,2.0,7.0,80,20.0


In [14]:
# Use LDA modeling to create occupation grouping

# create term-document matrix
# countvec = CountVectorizer()
# X = countvec.fit_transform(dem.OCCUPATION)

# # fit LDA model
# model = lda.LDA(n_topics=20, n_iter=1500, random_state=1)
# model.fit(X)

In [15]:
# topic_word = model.topic_word_ 
# n_top_words = 8
# for i, topic_dist in enumerate(topic_word):
#     topic_words = np.array(vocab)[np.argsort(topic_dist)][:-n_top_words:-1]
#     print('Topic {}: {}'.format(i, ' '.join(topic_words)))

In [16]:
# apply occupation grouping based on keywords
dem['OCCUPATION1'] = dem['OCCUPATION'].apply(group_occupation)
dem.OCCUPATION1.value_counts()

TECH                            1673
MISSING                         1113
EXECUTIVE                        694
SELF                             482
FINANCE                          386
MANAGER                          381
MEDICINE                         302
STUDENT                          275
SALES                            227
RETIRED                          206
CONSULTANT                       174
INVESTOR                         156
LAW                              149
REAL ESTATE                      147
TRADER                           118
ENTREPRENEUR                     104
BUSINESS                          86
MARKETING                         75
ANALYST                           56
UNEMPLOYED                        49
TEACHER                           35
DESIGNER                          22
PILOT                             21
PROFESSOR                         19
RESEARCHER                        19
ARTIST                            19
DRIVER                            18
C

In [17]:
# Group occupations that show up less than 10 times as 'OTHER'
dem['dummy'] = 1
occs = dem.groupby('OCCUPATION1').agg({'dummy':sum}).reset_index()
other_keywords = list(occs[occs['dummy'] < 10].OCCUPATION1.unique())
dem['OCCUPATION2'] = dem['OCCUPATION1'].apply(group_occupation, other=other_keywords)

In [18]:
# new occupation grouping
dem.OCCUPATION2.value_counts()

TECH            1673
OTHER           1155
MISSING         1113
EXECUTIVE        694
SELF             482
FINANCE          386
MANAGER          381
MEDICINE         302
STUDENT          275
SALES            227
RETIRED          206
CONSULTANT       174
INVESTOR         156
LAW              149
REAL ESTATE      147
TRADER           118
ENTREPRENEUR     104
BUSINESS          86
MARKETING         75
ANALYST           56
UNEMPLOYED        49
TEACHER           35
DESIGNER          22
PILOT             21
RESEARCHER        19
PROFESSOR         19
ARTIST            19
DRIVER            18
CONSTRUCTION      14
PHOTOGRAPHER      12
POKER PLAYER      11
INSURANCE         11
CONSULTING        11
MUSICIAN          11
BARTENDER         10
Name: OCCUPATION2, dtype: int64

In [19]:
dem.groupby('level').agg({'OCCUPATION2':lambda x:x.value_counts().to_dict()})

,OCCUPATION2
level,
level_a,{'MISSING': 82}
level_b,"{'TECH': 971, 'OTHER': 593, 'MISSING': 441, 'E..."
level_c,"{'TECH': 236, 'OTHER': 195, 'MISSING': 189, 'E..."
level_d,"{'TECH': 452, 'OTHER': 349, 'MISSING': 244, 'E..."
level_e,{'MISSING': 91}
level_f,"{'OTHER': 17, 'TECH': 14, 'MISSING': 7, 'EXECU..."
level_g,"{'MISSING': 46, 'STUDENT': 2, 'MANAGER': 2, 'O..."
level_h,{'MISSING': 12}
level_i,{'MISSING': 1}


In [20]:
dem.groupby('level').agg(lambda x:x.value_counts().to_dict())

,EXCHANGE_ACCOUNT_ID,CREATED_AT,FIRST_VERIFIED_AT,STATE_CODE,COUNTRY_CODE,BIRTH_YEAR,OCCUPATION,BROWSER_COUNT,OS_COUNT,TOTAL_DEVICES,SESSION_COUNT,COUNT_BANKS,OCCUPATION1,dummy,OCCUPATION2
level,,,,,,,,,,,,,,,
level_a,"{6327: 2, 3: 2, 3097: 2, 7201: 2, 16511: 1, 55...","{'1/0/00': 48, '4/5/18 0:00': 2, '1/10/18 0:00...","{'6/28/17 0:00': 2, '6/5/17 0:00': 2, '4/10/17...","{'NAN': 63, 'CA': 7, 'FL': 3, 'NY': 2, 'WA': 2...","{'NAN': 44, 'US': 20, 'CA': 4, 'IL': 2, 'HK': ...",{},{'NAN': 82},"{1.0: 49, 2.0: 6}","{1.0: 53, 2.0: 2}","{1.0: 42, 2.0: 12, 3.0: 1}","{1: 80, 3: 1, 2: 1}","{2.0: 16, 5.0: 13, 4.0: 12, 3.0: 11, 8.0: 7, 1...",{'MISSING': 82},{1: 82},{'MISSING': 82}
level_b,"{3477: 5, 2753: 4, 807: 4, 333: 4, 8453: 3, 22...","{'1/0/00': 48, '5/16/17 0:00': 32, '6/13/17 0:...","{'5/25/17 0:00': 55, '5/24/17 0:00': 42, '5/26...","{'CA': 923, 'NY': 569, 'FL': 390, 'TX': 347, '...","{'US': 4201, 'IS': 1, 'CA': 1, 'PR': 1}","{1982.0: 187, 1985.0: 178, 1984.0: 166, 1988.0...","{'NAN': 441, 'SOFTWARE ENGINEER': 166, 'SELF E...","{2.0: 1090, 3.0: 1042, 1.0: 866, 4.0: 645, 5.0...","{2.0: 1089, 3.0: 998, 1.0: 972, 4.0: 601, 5.0:...","{4.0: 308, 5.0: 299, 6.0: 289, 7.0: 279, 8.0: ...","{37: 30, 49: 30, 67: 28, 46: 28, 33: 27, 38: 2...","{2.0: 1237, 1.0: 1102, 3.0: 736, 4.0: 501, 5.0...","{'TECH': 971, 'MISSING': 441, 'EXECUTIVE': 345...",{1: 4204},"{'TECH': 971, 'OTHER': 593, 'MISSING': 441, 'E..."
level_c,"{5221: 5, 11907: 5, 6623: 4, 67: 4, 6309: 4, 1...","{'1/0/00': 48, '10/8/15 0:00': 15, '5/16/17 0:...","{'10/8/15 0:00': 20, '5/25/17 0:00': 16, '10/6...","{'CA': 218, 'NAN': 211, 'NY': 158, 'FL': 120, ...","{'US': 986, 'SG': 130, 'CA': 96, 'HK': 80, 'GB...","{1984.0: 66, 1988.0: 61, 1985.0: 55, 1983.0: 5...","{'NAN': 189, 'RETIRED': 52, 'STUDENT': 40, 'SE...","{2.0: 340, 3.0: 327, 1.0: 266, 4.0: 174, 5.0: ...","{3.0: 314, 1.0: 308, 2.0: 293, 4.0: 221, 5.0: ...","{5.0: 95, 4.0: 94, 7.0: 93, 3.0: 83, 6.0: 83, ...","{1: 15, 31: 11, 18: 10, 79: 10, 92: 10, 133: 9...","{2.0: 229, 3.0: 182, 4.0: 164, 5.0: 149, 1.0: ...","{'TECH': 236, 'MISSING': 189, 'EXECUTIVE': 127...",{1: 1374},"{'TECH': 236, 'OTHER': 195, 'MISSING': 189, 'E..."
level_d,"{1135: 10, 7025: 7, 9769: 5, 2799: 5, 1157: 5,...","{'1/0/00': 58, '5/16/17 0:00': 29, '6/12/17 0:...","{'10/5/15 0:00': 40, '5/25/17 0:00': 27, '5/24...","{'CA': 419, 'NAN': 388, 'NY': 251, 'FL': 173, ...","{'US': 1734, 'SG': 256, 'CA': 119, 'HK': 108, ...","{1985.0: 108, 1986.0: 100, 1984.0: 99, 1983.0:...","{'NAN': 244, 'STUDENT': 76, 'RETIRED': 75, 'SE...","{2.0: 571, 3.0: 560, 1.0: 432, 4.0: 409, 5.0: ...","{3.0: 569, 2.0: 520, 1.0: 493, 4.0: 375, 5.0: ...","{7.0: 157, 8.0: 153, 6.0: 152, 10.0: 144, 4.0:...","{1: 17, 48: 15, 2: 14, 23: 14, 31: 14, 33: 13,...","{2.0: 592, 3.0: 404, 4.0: 358, 1.0: 251, 5.0: ...","{'TECH': 452, 'MISSING': 244, 'EXECUTIVE': 215...",{1: 2339},"{'TECH': 452, 'OTHER': 349, 'MISSING': 244, 'E..."
level_e,"{3339: 5, 1855: 4, 10609: 3, 5103: 3, 2247: 2,...","{'1/0/00': 25, '1/15/18 0:00': 4, '2/27/17 0:0...","{'11/21/17 0:00': 5, '1/12/18 0:00': 4, '2/22/...","{'NAN': 16, 'CO': 14, 'IL': 10, 'CA': 10, 'FL'...","{'US': 75, 'CA': 7, 'SG': 5, 'KY': 1, 'BR': 1,...",{},{'NAN': 91},"{1.0: 50, 2.0: 15, 3.0: 2, 4.0: 1}","{1.0: 52, 2.0: 15, 3.0: 1}","{1.0: 34, 3.0: 9, 2.0: 8, 4.0: 5, 5.0: 5, 7.0:...","{1: 47, 2: 10, 3: 5, 4: 5, 5: 3, 12: 2, 22: 2,...","{2.0: 22, 3.0: 17, 4.0: 16, 5.0: 11, 6.0: 8, 1...",{'MISSING': 91},{1: 91},{'MISSING': 91}
level_f,"{427: 4, 12541: 2, 12295: 2, 4831: 2, 3545: 2,...","{'1/0/00': 41, '11/7/17 0:00': 2, '11/17/17 0:...","{'3/7/18 0:00': 4, '12/10/17 0:00': 3, '12/5/1...","{'NAN': 16, 'CA': 15, 'NY': 7, 'FL': 6, 'ON': ...","{'US': 48, 'SG': 14, 'HK': 7, 'KR': 6, 'CA': 5...","{1992.0: 6, 1986.0: 5, 1988.0: 5, 1990.0: 5, 1...","{'NAN': 7, 'ENTREPRENEUR': 4, 'CONSULTANT': 4,...","{1.0: 46, 2.0: 15, 3.0: 2}","{1.0: 47, 2.0: 12, 3.0: 4}","{1.0: 30, 2.0: 18, 3.0: 10, 4.0: 4, 6.0: 1}","{1: 25, 2: 22, 4: 8, 6: 5, 3: 5, 5: 4, 11: 3, ...","{2.0: 23, 3.0: 13, 4.0

In [21]:
# impute state code based on country code
dem.loc[(dem['STATE_CODE'] == 'NAN') & (dem['COUNTRY_CODE'] != 'NAN') & (dem['COUNTRY_CODE'] != 'US'), 'STATE_CODE'] = 'NON US'

In [22]:
dem.COUNTRY_CODE.value_counts()

US     7113
SG      413
CA      238
HK      199
GB      121
NAN      44
KR       38
IL       12
AU        7
PL        7
FR        6
BR        5
KY        5
JP        4
PH        3
MX        3
MU        3
TH        2
CN        2
VC        2
VG        2
LB        1
EE        1
CW        1
BE        1
ZA        1
PR        1
TR        1
BS        1
CH        1
ID        1
IE        1
IS        1
Name: COUNTRY_CODE, dtype: int64

In [23]:
dem.STATE_CODE.value_counts()

CA                         1603
NY                         1000
FL                          699
NON US                      662
TX                          529
IL                          312
NJ                          252
WA                          220
CO                          164
PA                          154
NC                          151
GA                          145
VA                          141
MA                          131
OH                          131
NV                          129
MI                          117
MN                          112
MD                          103
ON                          101
CT                           91
UT                           84
MO                           73
TN                           71
HONG KONG                    61
IN                           60
BC                           59
LA                           47
OR                           47
NAN                          45
                           ... 
SAI KUNG

In [24]:
now = datetime.datetime.now().year
dem['age'] = now - dem.BIRTH_YEAR
dem.age.describe()

count    8061.000000
mean       43.147128
std        12.050470
min        20.000000
25%        34.000000
50%        41.000000
75%        50.000000
max        98.000000
Name: age, dtype: float64

In [25]:
# convert to datetime
dem['CREATED_AT'] =  pd.to_datetime(dem['CREATED_AT'], errors='coerce')
dem['FIRST_VERIFIED_AT'] =  pd.to_datetime(dem['FIRST_VERIFIED_AT'], errors='coerce')
# how long did it take them to verify account
dem['days_til_verification'] = dem['CREATED_AT'] - dem['FIRST_VERIFIED_AT']
dem['days_til_verification'] = dem['days_til_verification'].apply(lambda x: x.days)
# how long ago did they create the account
dem['days_since_creation'] = datetime.datetime.now() - dem['CREATED_AT']
dem['days_since_creation'] = dem['days_since_creation'].apply(lambda x: x.days)

In [26]:
dem.head()

,EXCHANGE_ACCOUNT_ID,level,CREATED_AT,FIRST_VERIFIED_AT,STATE_CODE,COUNTRY_CODE,BIRTH_YEAR,OCCUPATION,BROWSER_COUNT,OS_COUNT,TOTAL_DEVICES,SESSION_COUNT,COUNT_BANKS,OCCUPATION1,dummy,OCCUPATION2,age,days_til_verification,days_since_creation
0,1,level_a,NaT,2016-12-14,NAN,NAN,NaN,NAN,NaN,NaN,NaN,1,6.0,MISSING,1,MISSING,NaN,NaN,NaN
1,3,level_a,NaT,2017-04-10,NAN,NAN,NaN,NAN,NaN,NaN,NaN,1,4.0,MISSING,1,MISSING,NaN,NaN,NaN
2,5,level_a,NaT,2017-05-02,NAN,NAN,NaN,NAN,NaN,NaN,NaN,1,3.0,MISSING,1,MISSING,NaN,NaN,NaN
3,7,level_a,NaT,2017-06-28,NAN,NAN,NaN,NAN,NaN,NaN,NaN,1,16.0,MISSING,1,MISSING,NaN,NaN,NaN
4,9,level_a,NaT,2017-06-28,NAN,NAN,NaN,NAN,NaN,NaN,NaN,1,1.0,MISSING,1,MISSING,NaN,NaN,NaN


### Exposure

In [27]:
exp = pd.read_csv(data_dir + 'exposure_train.csv')

In [28]:
eda(exp)

Shape:
(45700, 7)
-----------------------------
Sample:
       EXCHANGE_ACCOUNT_ID  SENT_INDIRECT_EXPOSURE  SENT_DIRECT_EXPOSURE  \
0                    15755                    7.61                   0.0   
1                    10761                   57.97                   0.0   
2                    15755                   14.04                   0.0   
45697                 6587                 -163.35                   0.0   
45698                 6587                    0.00                   0.0   
45699                10819                    0.00                   0.0   

       RECEIVED_INDIRECT_EXPOSURE  RECEIVED_DIRECT_EXPOSURE cluster_category  \
0                         1765.90                       0.0       category_1   
1                           38.47                       0.0       category_2   
2                            0.00                       0.0       category_1   
45697                      -13.98                       0.0      category_11   
45698      

In [29]:
exp.head()

,EXCHANGE_ACCOUNT_ID,SENT_INDIRECT_EXPOSURE,SENT_DIRECT_EXPOSURE,RECEIVED_INDIRECT_EXPOSURE,RECEIVED_DIRECT_EXPOSURE,cluster_category,cluster_name
0,15755,7.61,0.0,1765.90,0.0,category_1,service_1
1,10761,57.97,0.0,38.47,0.0,category_2,service_3
2,15755,14.04,0.0,0.00,0.0,category_1,service_4
3,8617,5.39,0.0,0.00,0.0,category_2,service_3
4,15755,6089.46,0.0,725.21,0.0,category_1,service_7


In [30]:
# multiple rows per person <- will need to aggregate
exp['dummy'] = 1
exp.groupby('EXCHANGE_ACCOUNT_ID').agg({'dummy':'sum'}).sort_values(by='dummy', ascending=False)

,dummy
EXCHANGE_ACCOUNT_ID,
6137,424
16541,417
17459,403
15489,343
21631,323
16869,319
10153,315
12231,301
16177,293


In [31]:
# look at example
exp[exp['EXCHANGE_ACCOUNT_ID'] == 17831]

,EXCHANGE_ACCOUNT_ID,SENT_INDIRECT_EXPOSURE,SENT_DIRECT_EXPOSURE,RECEIVED_INDIRECT_EXPOSURE,RECEIVED_DIRECT_EXPOSURE,cluster_category,cluster_name,dummy
27413,17831,11796.06,11796.06,0.0,0.0,category_6,service_18,1
27651,17831,298190.31,298190.31,0.0,0.0,category_1,service_50,1


In [32]:
# create dummy variables for cluster_category
exp = exp.merge(pd.get_dummies(exp[['cluster_category']]), left_index=True, right_index=True)
exp.head()

,EXCHANGE_ACCOUNT_ID,SENT_INDIRECT_EXPOSURE,SENT_DIRECT_EXPOSURE,RECEIVED_INDIRECT_EXPOSURE,RECEIVED_DIRECT_EXPOSURE,cluster_category,cluster_name,dummy,cluster_category_category_1,cluster_category_category_10,cluster_category_category_11,cluster_category_category_12,cluster_category_category_13,cluster_category_category_14,cluster_category_category_15,cluster_category_category_2,cluster_category_category_3,cluster_category_category_4,cluster_category_category_5,cluster_category_category_6,cluster_category_category_7,cluster_category_category_8,cluster_category_category_9
0,15755,7.61,0.0,1765.90,0.0,category_1,service_1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10761,57.97,0.0,38.47,0.0,category_2,service_3,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,15755,14.04,0.0,0.00,0.0,category_1,service_4,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,8617,5.39,0.0,0.00,0.0,category_2,service_3,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,15755,6089.46,0.0,725.21,0.0,category_1,service_7,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [33]:
# define aggregations
agg_dict = {
             'dummy':'sum', 
             'SENT_INDIRECT_EXPOSURE':'sum',
             'SENT_DIRECT_EXPOSURE':'sum',
             'RECEIVED_INDIRECT_EXPOSURE':'sum',
             'RECEIVED_DIRECT_EXPOSURE':'sum',
             'cluster_category':'nunique',
             'cluster_name':'nunique'
            }

# create dict to be used in agg so that I don't have to type all of the names
clust_dict = {}
for clust in exp.cluster_category.unique():
    clust_dict['cluster_category_' + clust] = 'sum'
    
# merge the dicts together
agg_dict.update(clust_dict)

agg_dict

{'dummy': 'sum',
 'SENT_INDIRECT_EXPOSURE': 'sum',
 'SENT_DIRECT_EXPOSURE': 'sum',
 'RECEIVED_INDIRECT_EXPOSURE': 'sum',
 'RECEIVED_DIRECT_EXPOSURE': 'sum',
 'cluster_category': 'nunique',
 'cluster_name': 'nunique',
 'cluster_category_category_1': 'sum',
 'cluster_category_category_2': 'sum',
 'cluster_category_category_3': 'sum',
 'cluster_category_category_4': 'sum',
 'cluster_category_category_5': 'sum',
 'cluster_category_category_6': 'sum',
 'cluster_category_category_7': 'sum',
 'cluster_category_category_8': 'sum',
 'cluster_category_category_9': 'sum',
 'cluster_category_category_10': 'sum',
 'cluster_category_category_11': 'sum',
 'cluster_category_category_12': 'sum',
 'cluster_category_category_13': 'sum',
 'cluster_category_category_14': 'sum',
 'cluster_category_category_15': 'sum'}

In [34]:
exp_agg = exp.groupby('EXCHANGE_ACCOUNT_ID').agg(agg_dict).rename(columns={
                                                'dummy':'exposure_count',
                                                 'SENT_INDIRECT_EXPOSURE':'SENT_INDIRECT_EXPOSURE_SUM',
                                                 'SENT_INDIRECT_EXPOSURE':'SENT_INDIRECT_EXPOSURE_SUM',
                                                 'SENT_DIRECT_EXPOSURE':'SENT_DIRECT_EXPOSURE_SUM',
                                                 'RECEIVED_INDIRECT_EXPOSURE':'RECEIVED_INDIRECT_EXPOSURE_SUM',
                                                 'RECEIVED_DIRECT_EXPOSURE':'RECEIVED_DIRECT_EXPOSURE_SUM',
                                                 'cluster_category':'cluster_category_count',
                                                 'cluster_name':'cluster_name_count'
                                                 }).reset_index()
exp_agg.head()

,EXCHANGE_ACCOUNT_ID,exposure_count,SENT_INDIRECT_EXPOSURE_SUM,SENT_DIRECT_EXPOSURE_SUM,RECEIVED_INDIRECT_EXPOSURE_SUM,RECEIVED_DIRECT_EXPOSURE_SUM,cluster_category_count,cluster_name_count,cluster_category_category_1,cluster_category_category_2,cluster_category_category_3,cluster_category_category_4,cluster_category_category_5,cluster_category_category_6,cluster_category_category_7,cluster_category_category_8,cluster_category_category_9,cluster_category_category_10,cluster_category_category_11,cluster_category_category_12,cluster_category_category_13,cluster_category_category_14,cluster_category_category_15
0,5203,9,63314.01,109.73,100884.85,0.00,3,9,7,1,0,0,0,1,0,0,0,0,0,0,0,0,0
1,5227,47,8018.39,6038.38,25617.44,1759.73,7,28,31,1,4,3,2,4,2,0,0,0,0,0,0,0,0
2,5243,47,57843.28,18055.57,48972.36,5977.58,11,47,27,4,1,2,1,2,2,3,2,0,1,2,0,0,0
3,5327,3,545409.39,512260.62,0.00,0.00,2,3,1,0,0,0,0,2,0,0,0,0,0,0,0,0,0
4,5329,33,7755.72,7716.89,241066.62,192840.29,7,32,20,1,1,0,2,2,6,0,0,0,0,1,0,0,0


In [35]:
# create averages
for col in [col for col in exp_agg.columns if '_SUM' in col]:
    exp_agg[col.replace('_SUM', "_AVG")] = exp_agg[col] / exp_agg['exposure_count']
    
exp_agg.head()

,EXCHANGE_ACCOUNT_ID,exposure_count,SENT_INDIRECT_EXPOSURE_SUM,SENT_DIRECT_EXPOSURE_SUM,RECEIVED_INDIRECT_EXPOSURE_SUM,RECEIVED_DIRECT_EXPOSURE_SUM,cluster_category_count,cluster_name_count,cluster_category_category_1,cluster_category_category_2,cluster_category_category_3,cluster_category_category_4,cluster_category_category_5,cluster_category_category_6,cluster_category_category_7,cluster_category_category_8,cluster_category_category_9,cluster_category_category_10,cluster_category_category_11,cluster_category_category_12,cluster_category_category_13,cluster_category_category_14,cluster_category_category_15,SENT_INDIRECT_EXPOSURE_AVG,SENT_DIRECT_EXPOSURE_AVG,RECEIVED_INDIRECT_EXPOSURE_AVG,RECEIVED_DIRECT_EXPOSURE_AVG
0,5203,9,63314.01,109.73,100884.85,0.00,3,9,7,1,0,0,0,1,0,0,0,0,0,0,0,0,0,7034.890000,12.192222,11209.427778,0.000000
1,5227,47,8018.39,6038.38,25617.44,1759.73,7,28,31,1,4,3,2,4,2,0,0,0,0,0,0,0,0,170.604043,128.476170,545.051915,37.441064
2,5243,47,57843.28,18055.57,48972.36,5977.58,11,47,27,4,1,2,1,2,2,3,2,0,1,2,0,0,0,1230.708085,384.161064,1041.965106,127.182553
3,5327,3,545409.39,512260.62,0.00,0.00,2,3,1,0,0,0,0,2,0,0,0,0,0,0,0,0,0,181803.130000,170753.540000,0.000000,0.000000
4,5329,33,7755.72,7716.89,241066.62,192840.29,7,32,20,1,1,0,2,2,6,0,0,0,0,1,0,0,0,235.021818,233.845152,7305.049091,5843.645152


In [36]:
# get sums at account/category level
out = pd.pivot_table(data=exp, index=exp.EXCHANGE_ACCOUNT_ID, columns=exp.cluster_category, aggfunc='sum')
out.columns = out.columns.to_series().str.join('_')
out = out.reset_index()
out.head()

,EXCHANGE_ACCOUNT_ID,RECEIVED_DIRECT_EXPOSURE_category_1,RECEIVED_DIRECT_EXPOSURE_category_10,RECEIVED_DIRECT_EXPOSURE_category_11,RECEIVED_DIRECT_EXPOSURE_category_12,RECEIVED_DIRECT_EXPOSURE_category_13,RECEIVED_DIRECT_EXPOSURE_category_14,RECEIVED_DIRECT_EXPOSURE_category_15,RECEIVED_DIRECT_EXPOSURE_category_2,RECEIVED_DIRECT_EXPOSURE_category_3,RECEIVED_DIRECT_EXPOSURE_category_4,RECEIVED_DIRECT_EXPOSURE_category_5,RECEIVED_DIRECT_EXPOSURE_category_6,RECEIVED_DIRECT_EXPOSURE_category_7,RECEIVED_DIRECT_EXPOSURE_category_8,RECEIVED_DIRECT_EXPOSURE_category_9,RECEIVED_INDIRECT_EXPOSURE_category_1,RECEIVED_INDIRECT_EXPOSURE_category_10,RECEIVED_INDIRECT_EXPOSURE_category_11,RECEIVED_INDIRECT_EXPOSURE_category_12,RECEIVED_INDIRECT_EXPOSURE_category_13,RECEIVED_INDIRECT_EXPOSURE_category_14,RECEIVED_INDIRECT_EXPOSURE_category_15,RECEIVED_INDIRECT_EXPOSURE_category_2,RECEIVED_INDIRECT_EXPOSURE_category_3,RECEIVED_INDIRECT_EXPOSURE_category_4,RECEIVED_INDIRECT_EXPOSURE_category_5,RECEIVED_INDIRECT_EXPOSURE_category_6,RECEIVED_INDIRECT_EXPOSURE_category_7,RECEIVED_INDIRECT_EXPOSURE_category_8,RECEIVED_INDIRECT_EXPOSURE_category_9,SENT_DIRECT_EXPOSURE_category_1,SENT_DIRECT_EXPOSURE_category_10,SENT_DIRECT_EXPOSURE_category_11,SENT_DIRECT_EXPOSURE_category_12,SENT_DIRECT_EXPOSURE_category_13,SENT_DIRECT_EXPOSURE_category_14,SENT_DIRECT_EXPOSURE_category_15,SENT_DIRECT_EXPOSURE_category_2,SENT_DIRECT_EXPOSURE_category_3,SENT_DIRECT_EXPOSURE_category_4,SENT_DIRECT_EXPOSURE_category_5,SENT_DIRECT_EXPOSURE_category_6,SENT_DIRECT_EXPOSURE_category_7,SENT_DIRECT_EXPOSURE_category_8,SENT_DIRECT_EXPOSURE_category_9,SENT_INDIRECT_EXPOSURE_category_1,SENT_INDIRECT_EXPOSURE_category_10,SENT_INDIRECT_EXPOSURE_category_11,SENT_INDIRECT_EXPOSURE_category_12,SENT_INDIRECT_EXPOSURE_category_13,SENT_INDIRECT_EXPOSURE_category_14,SENT_INDIRECT_EXPOSURE_category_15,SENT_INDIRECT_EXPOSURE_category_2,SENT_INDIRECT_EXPOSURE_category_3,SENT_INDIRECT_EXPOSURE_category_4,SENT_INDIRECT_EXPOSURE_category_5,SENT_INDIRECT_EXPOSURE_category_6,SENT_INDIRECT_EXPOSURE_category_7,SENT_INDIRECT_EXPOSURE_category_8,SENT_INDIRECT_EXPOSURE_category_9,cluster_category_category_1_category_1,cluster_category_category_1_category_10,cluster_category_category_1_category_11,cluster_category_category_1_category_12,cluster_category_category_1_category_13,cluster_category_category_1_category_14,cluster_category_category_1_category_15,cluster_category_category_1_category_2,cluster_category_category_1_category_3,cluster_category_category_1_category_4,cluster_category_category_1_category_5,cluster_category_category_1_category_6,cluster_category_category_1_category_7,cluster_category_category_1_category_8,cluster_category_category_1_category_9,cluster_category_category_10_category_1,cluster_category_category_10_category_10,cluster_category_category_10_category_11,cluster_category_category_10_category_12,cluster_category_category_10_category_13,cluster_category_category_10_category_14,cluster_category_category_10_category_15,cluster_category_category_10_category_2,cluster_category_category_10_category_3,cluster_category_category_10_category_4,cluster_category_category_10_category_5,cluster_category_category_10_category_6,cluster_category_category_10_category_7,cluster_category_category_10_category_8,cluster_category_category_10_category_9,cluster_category_category_11_category_1,cluster_category_category_11_category_10,cluster_category_category_11_category_11,cluster_category_category_11_category_12,cluster_category_category_11_category_13,cluster_category_category_11_category_14,cluster_category_category_11_category_15,cluster_category_category_11_category_2,cluster_category_category_11_category_3,cluster_category_category_11_category_4,cluster_category_category_11_category_5,cluster_category_category_11_category_6,cluster_category_category_11_category_7,cluster_category_category_11_category_8,cluster_category_category_11_category_9,cluster_category_category_12_category_1,cluster_cate

### Transfers

In [37]:
transfers = pd.read_csv(data_dir + 'transfers_train.csv', low_memory=False)

In [38]:
eda(transfers)

Shape:
(1043311, 5)
-----------------------------
Sample:
         ACCOUNT_ID      TX_TIME    CURRENCY  AMOUNT    TYPE
0            9035.0  9/7/17 1:10  currency_2    2.70  type_2
1            7527.0  9/7/17 1:13  currency_2    0.50  type_2
2           18305.0  9/7/17 1:10  currency_1  757.08  type_2
1043308         NaN          NaN         NaN     NaN     NaN
1043309         NaN          NaN         NaN     NaN     NaN
1043310         NaN          NaN         NaN     NaN     NaN
-----------------------------
Describe:
          ACCOUNT_ID        TX_TIME    CURRENCY        AMOUNT    TYPE
count   14736.000000          14736       14736  1.473600e+04   14736
unique           NaN           7472           6           NaN       5
top              NaN  9/12/17 21:07  currency_2           NaN  type_3
freq             NaN            229        6461           NaN    6178
mean    12805.587676            NaN         NaN  4.261884e+03     NaN
std      4883.351185            NaN         NaN  3.6819

In [39]:
transfers.head().append(transfers.tail())

,ACCOUNT_ID,TX_TIME,CURRENCY,AMOUNT,TYPE
0,9035.0,9/7/17 1:10,currency_2,2.700,type_2
1,7527.0,9/7/17 1:13,currency_2,0.500,type_2
2,18305.0,9/7/17 1:10,currency_1,757.080,type_2
3,7527.0,9/7/17 1:12,currency_3,0.886,type_2
4,6545.0,9/7/17 1:14,currency_1,500.000,type_3
1043306,NaN,NaN,NaN,NaN,NaN
1043307,NaN,NaN,NaN,NaN,NaN
1043308,NaN,NaN,NaN,NaN,NaN
1043309,NaN,NaN,NaN,NaN,NaN
1043310,NaN,NaN,NaN,NaN,NaN


In [40]:
# how many rows are missing all data? <- most of it!! (98%)
print('Bad rows:')
print(transfers.isnull().all(1).sum())
transfers[transfers.isnull().all(1)].head()

Bad rows:
1028575


,ACCOUNT_ID,TX_TIME,CURRENCY,AMOUNT,TYPE
14736,NaN,NaN,NaN,NaN,NaN
14737,NaN,NaN,NaN,NaN,NaN
14738,NaN,NaN,NaN,NaN,NaN
14739,NaN,NaN,NaN,NaN,NaN
14740,NaN,NaN,NaN,NaN,NaN


In [41]:
# drop rows with all nans
transfers.dropna(axis = 0, how = 'all', inplace = True)

In [42]:
# convert to int
transfers = transfers.astype({'ACCOUNT_ID':np.int64})
# convert to datetime
transfers['TX_TIME'] =  pd.to_datetime(transfers['TX_TIME'], errors='coerce')

In [43]:
transfers.head()

,ACCOUNT_ID,TX_TIME,CURRENCY,AMOUNT,TYPE
0,9035,2017-09-07 01:10:00,currency_2,2.700,type_2
1,7527,2017-09-07 01:13:00,currency_2,0.500,type_2
2,18305,2017-09-07 01:10:00,currency_1,757.080,type_2
3,7527,2017-09-07 01:12:00,currency_3,0.886,type_2
4,6545,2017-09-07 01:14:00,currency_1,500.000,type_3


In [44]:
# look at it again
eda(transfers)

Shape:
(14736, 5)
-----------------------------
Sample:
       ACCOUNT_ID             TX_TIME    CURRENCY     AMOUNT    TYPE
0            9035 2017-09-07 01:10:00  currency_2    2.70000  type_2
1            7527 2017-09-07 01:13:00  currency_2    0.50000  type_2
2           18305 2017-09-07 01:10:00  currency_1  757.08000  type_2
14733       10257 2018-01-18 22:23:00  currency_1  500.00000  type_3
14734        7517 2018-01-10 00:04:00  currency_2    0.76604  type_2
14735        8881 2018-01-10 00:06:00  currency_1  500.00000  type_3
-----------------------------
Describe:
          ACCOUNT_ID              TX_TIME    CURRENCY        AMOUNT    TYPE
count   14736.000000                14736       14736  1.473600e+04   14736
unique           NaN                 7472           6           NaN       5
top              NaN  2017-09-12 21:07:00  currency_2           NaN  type_3
freq             NaN                  229        6461           NaN    6178
first            NaN  2017-09-07 01:10:00

In [45]:
# will need to aggregate
transfers.groupby('ACCOUNT_ID').agg('count').sort_values(by='TX_TIME', ascending=False)

,TX_TIME,CURRENCY,AMOUNT,TYPE
ACCOUNT_ID,,,,
8673,153,153,153,153
11507,141,141,141,141
9533,114,114,114,114
15327,72,72,72,72
6281,69,69,69,69
17301,66,66,66,66
11217,57,57,57,57
12295,56,56,56,56
7295,56,56,56,56


In [46]:
# aggregate to account level and create features broken out by category
transfer_agg = pd.pivot_table(data=transfers, index=transfers.ACCOUNT_ID, columns=transfers.TYPE, aggfunc=('sum','nunique'))
transfer_agg.columns = transfer_agg.columns.to_series().str.join('_')
transfer_agg = transfer_agg.reset_index()
transfer_agg.head()

,ACCOUNT_ID,AMOUNT_nunique_type_1,AMOUNT_nunique_type_2,AMOUNT_nunique_type_3,AMOUNT_nunique_type_4,AMOUNT_nunique_type_5,AMOUNT_sum_type_1,AMOUNT_sum_type_2,AMOUNT_sum_type_3,AMOUNT_sum_type_4,AMOUNT_sum_type_5,CURRENCY_nunique_type_1,CURRENCY_nunique_type_2,CURRENCY_nunique_type_3,CURRENCY_nunique_type_4,CURRENCY_nunique_type_5,CURRENCY_sum_type_1,CURRENCY_sum_type_2,CURRENCY_sum_type_3,CURRENCY_sum_type_4,CURRENCY_sum_type_5
0,5203,NaN,NaN,1.0,NaN,NaN,NaN,NaN,500.000000,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,currency_1,NaN,NaN
1,5205,NaN,1.0,NaN,NaN,NaN,NaN,4264.650000,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,currency_1,NaN,NaN,NaN
2,5207,1.0,NaN,1.0,NaN,NaN,1000.0,NaN,1000.000000,NaN,NaN,1.0,NaN,1.0,NaN,NaN,currency_1currency_1,NaN,currency_1currency_1,NaN,NaN
3,5209,NaN,11.0,3.0,NaN,NaN,NaN,149.289715,254000.000000,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,currency_2currency_2currency_2currency_2curren...,currency_1currency_1currency_1,NaN,NaN
4,5211,NaN,NaN,1.0,NaN,NaN,NaN,NaN,94.999559,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,currency_3,NaN,NaN
